In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import linalg
import os, sys
import matplotlib.pyplot as plt

pucks = pd.read_csv('./pucks.csv')
tickets = pd.read_csv('./tickets.csv')
gates = pd.read_csv('./gates.csv')

print('Okay')

Okay


In [2]:
print('------------------PUCKS------------------')
print(pucks.info())
print('-----------------------------------------')
print(pucks.describe())
print('-----------------------------------------')

------------------PUCKS------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 18 columns):
No                 753 non-null object
ArrivingDate       753 non-null object
ArrivingTime       753 non-null object
ArrivingFlight     753 non-null object
ArrivingDI         753 non-null object
FlightType         753 non-null object
DepartDate         753 non-null object
DepartTime         753 non-null object
DepartFlight       753 non-null object
DepartDI           753 non-null object
PreviousAirport    753 non-null object
NextAirport        753 non-null object
BinaryWN           753 non-null object
ArrivingDay        753 non-null int64
DepartDay          753 non-null int64
ArrivingTimeIdx    753 non-null int64
DepartTimeIdx      753 non-null int64
TimeDiff           753 non-null int64
dtypes: int64(5), object(13)
memory usage: 106.0+ KB
None
-----------------------------------------
       ArrivingDay   DepartDay  ArrivingTimeIdx  DepartT

In [3]:
print('------------------TICKETS------------------')
print(tickets.info())
print('-----------------------------------------')
print(tickets.describe())
print('-----------------------------------------')

------------------TICKETS------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4796 entries, 0 to 4795
Data columns (total 6 columns):
PsgID             4796 non-null object
PsgNumber         4796 non-null int64
ArrivingFlight    4796 non-null object
ArrivingDate      4796 non-null object
DepartFlight      4796 non-null object
DepartDate        4796 non-null object
dtypes: int64(1), object(5)
memory usage: 224.9+ KB
None
-----------------------------------------
         PsgNumber
count  4796.000000
mean      1.669725
std       0.470361
min       1.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       2.000000
-----------------------------------------


In [4]:
print('------------------GATES------------------')
print(gates.info())
print('-----------------------------------------')
print(gates.describe())
print('-----------------------------------------')

------------------GATES------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 6 columns):
Gate          69 non-null object
Terminal      69 non-null object
Region        69 non-null object
ArrivingDI    69 non-null object
DepartDI      69 non-null object
FlightType    69 non-null object
dtypes: object(6)
memory usage: 3.3+ KB
None
-----------------------------------------
       Gate Terminal  Region ArrivingDI DepartDI FlightType
count    69       69      69         69       69         69
unique   69        2       4          3        3          2
top     S40        S  Center          D        D          N
freq      1       41      20         39       39         45
-----------------------------------------


In [11]:
# Pucks表数据预处理
wide_bodies = set(['332', '333', '33E', '33H', '33L', '773'])
narrow_bodies = set(['319', '320', '321', '323', '325', '738', '73A', '73E', '73H', '73L'])
        

def convert_wn_binary(item):
    if type(item) is not str:
        item = str(item)
    if item in wide_bodies:
        return 'W'
    elif item in narrow_bodies:
        return 'N'
    else:
        raise Exception('Invalid argument')
        
def convert_dates(item):
    if type(item) is not str:
        item = str(item)
    return int(item.split('-')[0])

def convert_timeidx(times):
    times = str(times)
    stimes = times.split(':')
    hour, minute = [int(stimes[0]), int(stimes[1])]
    return hour * 12 + (minute / 5)

        
wn_binaries = pucks['FlightType'].map(convert_wn_binary)
arv_days = pucks['ArrivingDate'].map(convert_dates)
depart_days = pucks['DepartDate'].map(convert_dates)
arv_timeidx = pucks['ArrivingTime'].map(convert_timeidx)
depart_timeidx = pucks['DepartTime'].map(convert_timeidx)

pucks['BinaryWN'] = wn_binaries
pucks['ArrivingDay'] = arv_days
pucks['DepartDay'] = depart_days
pucks['ArrivingTimeIdx'] = arv_timeidx
pucks['DepartTimeIdx'] = depart_timeidx
pucks.to_csv('pucks.csv')
print(pucks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 17 columns):
No                 753 non-null object
ArrivingDate       753 non-null object
ArrivingTime       753 non-null object
ArrivingFlight     753 non-null object
ArrivingDI         753 non-null object
FlightType         753 non-null object
DepartDate         753 non-null object
DepartTime         753 non-null object
DepartFlight       753 non-null object
DepartDI           753 non-null object
PreviousAirport    753 non-null object
NextAirport        753 non-null object
BinaryWN           753 non-null object
ArrivingDay        753 non-null int64
DepartDay          753 non-null int64
ArrivingTimeIdx    753 non-null float64
DepartTimeIdx      753 non-null float64
dtypes: float64(2), int64(2), object(13)
memory usage: 100.1+ KB
None


In [191]:
# 匹配查询功能函数

class GateInfo(object):
    def __init__(self, gatedf, curtime=0):
        self.usage = dict()
        self.time_table = dict()
        self.hashing = dict()
        self.belonging = dict()
        for item in gatedf['Gate']:
            self.usage[str(item)] = False
            self.belonging[str(item)] = ''
            self.time_table[str(item)] = ()
            self.hashing[str(item)] = False
        self.df = gatedf
        self.curtime = curtime
        
    def update_time(self, time):
        '''
        :param ctime: integer, current time index
        '''
        for gt, usag in self.usage.items():
            if usag and self.time_table[gt][1] + 9 <= time:
                self.usage[gt] = False
        self.curtime = time
        
    def robin_hood(self, query):
        '''
        :param query: class Query
        :return 1: which flight is being robbed
        :return 2: which gate to rob
         In order to reduce the possibility of failure, rob any gates available if possible
        '''
        self.get_available(query.arv_di, query.dpt_di, query.wn)
        selected = self.df.loc[self.df['s1']==1].loc[self.df['s2']==1].loc[self.df['s3']==1]['Gate'].values.tolist()
#         print(selected)
        for gn in selected:
            if self.time_table[gn][1] >= (query.end + 9):
                return self.belonging[gn], gn
        return None, None
        
    def update_gates(self, gname, fname, end_t):
        '''
        :param gname: type str, gate name
        :param fname: type str, flight name
        :param end_t: integer time index
        '''
        if not self.usage[gname]:
            self.usage[gname] = True
            self.hashing[gname] = True
            self.belonging[gname] = fname
            self.time_table[gname] = (self.curtime, end_t)
        else:
            raise Exception('Attempting to use an unavailable port!')

    def get_available(self, arv_di, depart_di, binary_wn):
        '''
        :param arv_di: type str, either 'D' or 'I'
        :param depart_di: type str, either 'D' or 'I'
        :param binary_wn: type str, either 'W' or 'N'
        :return: list containing all the names of the available gates
         Note that the returned list could be empty if there is no available port
        '''
        def get_maps(_arv_di, _depart_di, _flight_type):
            return 1 if arv_di in str(arv_di) and depart_di in str(_depart_di) and flight_type == binary_wn else 0
        
        self.df['s1'] = self.df['ArrivingDI'].map(lambda x: 1 if arv_di in str(x) else 0)
        self.df['s2'] = self.df['DepartDI'].map(lambda x: 1 if depart_di in str(x) else 0)
        self.df['s3'] = self.df['FlightType'].map(lambda x: 1 if binary_wn==x else 0)
        selected = self.df.loc[self.df['s1']==1].loc[self.df['s2']==1].loc[self.df['s3']==1]['Gate'].values.tolist()
        n_selected = filter(lambda x: self.hashing[x] is True and self.usage[x] is False, selected)
        if len(n_selected) == 0:
            n_selected = filter(lambda x: not self.usage[x], selected)
        return n_selected
    
    def __str__(self):
        border = '----------------------------------------------------------- \n'
        for gt, usag in self.usage.items():
            border = border + gt + ': '
            if usag:
                begin_t = self.time_table[gt][0]; end_t = self.time_table[gt][1]
                begin_t %= 288; end_t %= 288
                begin_s = str(begin_t / 12) + ':' + str((begin_t % 12) * 5)
                end_s = str(end_t / 12) + ':' + str((end_t % 12) * 5)
                border += '--status: being used --flight: {} --begin time: {} --end time: {} \n'.\
                          format(self.belonging[gt], begin_s, end_s)
            else:
                border += '--status: available \n'
        border += '-----------------------------------------------------------'
        return border
    
def test_gate_info():
    info_test = GateInfo(gates)
    for tidx in range(0, 500, 2):
        info_test.update_time(tidx)
        res = info_test.get_available(np.random.choice(['D', 'I']), 
                                      np.random.choice(['D', 'I']),
                                      np.random.choice(['W', 'N']))
        if not res:
            print('Time {}. No available gate!!'.format(tidx))
            continue
        info_test.update_gates(res[0], 'ANONYMOUS', tidx + 15)
    print(info_test)
    
test_gate_info()

Time 22. No available gate!!
Time 70. No available gate!!
Time 80. No available gate!!
Time 110. No available gate!!
Time 132. No available gate!!
Time 146. No available gate!!
Time 150. No available gate!!
Time 192. No available gate!!
Time 194. No available gate!!
Time 212. No available gate!!
Time 220. No available gate!!
Time 240. No available gate!!
Time 246. No available gate!!
Time 274. No available gate!!
Time 312. No available gate!!
Time 324. No available gate!!
Time 346. No available gate!!
Time 364. No available gate!!
Time 372. No available gate!!
Time 382. No available gate!!
Time 392. No available gate!!
Time 412. No available gate!!
Time 438. No available gate!!
Time 440. No available gate!!
Time 442. No available gate!!
----------------------------------------------------------- 
S9: --status: available 
S8: --status: available 
S3: --status: available 
S2: --status: available 
S1: --status: available 
S36: --status: available 
S7: --status: available 
S6: --status: av

In [192]:
# 功能：获取给定时间点的所有航班

class FlightQuery(object):
    def __init__(self, puckdf):
        self.df = puckdf
        self.timeidx = list(set(puckdf['ArrivingTimeIdx'].values.tolist()))
        self.curtime = 0
        
    def next_point(self):
        critical_columns = ['No', 'ArrivingTimeIdx', 'DepartTimeIdx', 'BinaryWN', 'ArrivingDI', 'DepartDI']
        if self.curtime < len(self.timeidx):
            ctime = self.timeidx[self.curtime]
            selected = self.df.loc[self.df['ArrivingTimeIdx'] == ctime][critical_columns].values.tolist()
            self.curtime += 1
            return selected, ctime
        else:
            return None, None
        
def test_flight_query():
    query = FlightQuery(pucks)
    flight = query.next_point()
    print(flight)
    while flight is not None:
        flight = query.next_point()
    print('Finished!')
    
# test_flight_query()

In [204]:
# 执行单次规划过程

import random
from Queue import Queue, PriorityQueue

class Query(object):
    def __init__(self, query):
        self.name = query[0]
        self.begin = query[1]
        self.end = query[2]
        self.wn = query[3]
        self.arv_di = query[4]
        self.dpt_di = query[5]
        
    def __lt__(self, q):
        return self.end < q.end
        
    
def program_once(epsilon=0.01):
    query = FlightQuery(pucks)
    gate_info = GateInfo(gates)
    que = Queue()
    assignment = dict(); loss = 0
    
    flight, timeidx = query.next_point()
    while flight is not None:
        gate_info.update_time(timeidx)
        # 将新来的航班放入服务队列中
        for f in flight:
            que.put(Query(f))
        # 队列元素逐一出队，每个元素有1-epsilon的概率得到服务
        for _ in range(que.qsize()):
            f = que.get()
            if f.end < timeidx:
                # 若出现分配失败的航班，则尽可能通过剥夺方式进行分配
                robbed, gatename = gate_info.robin_hood(f)
                if robbed is not None:
                    assignment[f.name] = [gatename, f.begin, f.end]
                    assignment[robbed][1] = f.end + 9
                    continue
                else:
                    assignment[f.name] = list()
                    continue
            prob = random.uniform(0.0, 1.0)
            if prob > epsilon:
                ports = gate_info.get_available(f.arv_di, f.dpt_di, f.wn)
                if len(ports) == 0:
                    que.put(f)
                    continue
                assignment[f.name] = [ports[0], timeidx, f.end]
                gate_info.update_gates(ports[0], f.name, f.end)
            else:
                que.put(f)
        loss += que.qsize()
        flight, timeidx = query.next_point()
    return loss, assignment


loss_val, assignment = program_once()
print(loss_val, assignment)
print('Total number of failures {}'.format(len(filter(lambda x: len(x)==0, assignment.values()))))

(3113, {'PK620': ['T11', 802L, 837L], 'PK621': ['S13', 807L, 816L], 'PK622': ['T5', 810L, 812L], 'PK623': ['T12', 803L, 830L], 'PK624': ['T7', 843L, 854L], 'PK625': ['T8', 820L, 828L], 'PK626': ['S2', 806L, 832L], 'PK627': ['T9', 822L, 833L], 'PK628': ['T15', 807L, 833L], 'PK629': ['T24', 811L, 818L], 'PK239': ['T23', 112L, 138L], 'PK238': ['T20', 123L, 123L], 'PK610': ['T9', 795L, 813L], 'PK233': ['T18', 105L, 124L], 'PK232': ['T17', 105L, 167L], 'PK231': ['T16', 104L, 117L], 'PK230': ['T15', 104L, 118L], 'PK237': ['T7', 120L, 127L], 'PK236': ['S1', 107L, 119L], 'PK235': ['T21', 107L, 120L], 'PK234': ['T19', 107L, 127L], 'PK655': ['T10', 825L, 835L], 'PK633': ['T7', 811L, 829L], 'PK616': ['T21', 803L, 816L], 'PK654': ['S16', 824L, 834L], 'PK588': ['T20', 792L, 792L], 'PK307': ['S9', 467L, 487L], 'PK306': ['S8', 467L, 481L], 'PK305': ['S3', 467L, 479L], 'PK304': ['T2', 466L, 482L], 'PK303': ['T1', 474L, 483L], 'PK302': ['S12', 466L, 522L], 'PK301': ['T21', 468L, 494L], 'PK300': ['S7', 